# 1. Set up the notebook

Import modules.

In [4]:
import json
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.signal import find_peaks
from IPython.display import display, IFrame, HTML

Define a function to load flight data from hardware experiments and resample it at 100 Hz using linear interpolation.

In [5]:
def load_hardware_data(filename, t_min_offset=0, t_max_offset=0):
    # load raw data
    with open(filename, 'r') as f:
        data = json.load(f)

    # convert lists to numpy arrays
    for val in data.values():
        for key in val.keys():
            val[key] = np.array(val[key])

    # create an array of times at which to subsample
    t_min = -np.inf
    t_max = np.inf
    for key, val in data.items():
        t_min = max(t_min, val['time'][0])
        t_max = min(t_max, val['time'][-1])
    t_min += t_min_offset * 1000
    t_max -= t_max_offset * 1000
    nt = int(1 + np.floor((t_max - t_min) / 10.))
    t = np.arange(0, 10 * nt, 10) / 1000.
    resampled_data = {'time': t}

    # resample raw data with linear interpolation
    for k, v in data.items():
        f = interp1d((v['time'] - t_min) / 1000., v['data'])
        resampled_data[k] = f(t)
        
    # return the resampled data
    return resampled_data

# 2. Define constants

Define the acceleration of gravity in $\text{kg}\cdot\text{m} \;/\; \text{s}^2$:

In [6]:
g = 9.81

Define the mass of the drone in $\text{kg}$:

In [7]:
m = 0.0440

# 3. Estimate the moment of inertia about the $x$-axis

In [16]:
r = 0.03

In [17]:
data = load_hardware_data(
    'x_data.json',
    t_min_offset=0.,
    t_max_offset=0.
)

t = data['time']
w_x = np.deg2rad(data['gyro.x'])
w_y = np.deg2rad(data['gyro.y'])
w_z = np.deg2rad(data['gyro.z'])

# Find the index of each peak (increase "prominence" if you get bad results)
peaks = find_peaks(w_x, prominence=0)
i_peaks = peaks[0]

# Find the time at each peak
t_peaks = t[i_peaks]

# Find w_x at each peak (for visualization)
w_x_peaks = w_x[i_peaks]

t_diff = t_peaks[1:] - t_peaks[:-1]
t_diff = t_diff[10:]
# Find the mean difference as an estimate of the oscillation period
T = np.mean(t_diff)

In [18]:
J_x = (m * g * r/((2 * np.pi / T) ** 2)) - m * r ** 2 # <-- FIXME
print(J_x)

1.8260446760861986e-05


# 4. Estimate the moment of inertia about the $y$-axis

In [19]:
r = 0.03

In [20]:
data = load_hardware_data(
    'y_data.json',
    t_min_offset=0.,
    t_max_offset=0.
)
t = data['time']
w_x = np.deg2rad(data['gyro.x'])
w_y = np.deg2rad(data['gyro.y'])
w_z = np.deg2rad(data['gyro.z'])

# Find the index of each peak (increase "prominence" if you get bad results)
peaks = find_peaks(w_y, prominence=0)
i_peaks = peaks[0]

# Find the time at each peak
t_peaks = t[i_peaks]

t_diff = t_peaks[1:] - t_peaks[:-1]
t_diff = t_diff[10:]
# Find the mean difference as an estimate of the oscillation period
T = np.mean(t_diff)

In [21]:
J_y = (m * g * r/((2 * np.pi / T) ** 2)) - m * r ** 2 # <-- FIXME
print(J_y)

7.457926030303888e-05


# 5. Estimate the moment of inertia about the $z$-axis

In [22]:
r = 0.015

In [23]:
data = load_hardware_data(
    'z_data.json',
    t_min_offset=0.,
    t_max_offset=0.
)
t = data['time']
w_x = np.deg2rad(data['gyro.x'])
w_y = np.deg2rad(data['gyro.y'])
w_z = np.deg2rad(data['gyro.z'])

# Find the index of each peak (increase "prominence" if you get bad results)
peaks = find_peaks(w_z, prominence=0.025)
i_peaks = peaks[0]

# Find the time at each peak
t_peaks = t[i_peaks]

t_diff = t_peaks[1:] - t_peaks[:-1]
# Find the mean difference as an estimate of the oscillation period
T = np.mean(t_diff)

In [24]:
J_z = (m * g * r/((2 * np.pi / T) ** 2)) - m * r ** 2 # <-- FIXME
print(J_z)

4.5815682178722596e-05
